In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading the data

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torchvision.models as models
import warnings
# Ignore all warnings
warnings.filterwarnings('ignore')

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv',dtype=np.float32)
train


In [ ]:
n_train = len(train)
n_pixels = len(train.columns) - 1
n_class = len(set(train['label']))

print('Number of training samples: {0}'.format(n_train))
print('Number of training pixels: {0}'.format(n_pixels))
print('Number of classes: {0}'.format(n_class))

In [ ]:
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv',dtype=np.float32)
test

In [ ]:
n_test = len(test)
n_pixels = len(test.columns)

print('Number of test samples: {0}'.format(n_test))
print('Number of test pixels: {0}'.format(n_pixels))

For below code, X_train = X_train / 255 is dividing the training data set by 255. This is likely because the image data is typically represented by pixel values ranging from 0 to 255, with 0 being black and 255 being white. Normalizing the pixel values to be between 0 and 1 can make it easier for the model to converge during training.

 X_test = test / 255 is performing the same normalization on the test data set.

X_train = X_train.values.reshape(-1,28,28,1) is reshaping the training data set. The -1 argument is telling NumPy to infer the size of the first dimension of the array based on the size of the other dimensions. In this case, the other dimensions are 28 and 28, which correspond to the width and height of the images. The 1 in the last dimension corresponds to the number of color channels in the image, which is one since the images are likely grayscale. This reshaping operation is necessary because most machine learning frameworks expect input data to be in a specific format, typically a 4-dimensional tensor of shape (batch_size, height, width, channels).

In [ ]:
y_train = train.label.values
x_train = train.loc[:, train.columns != 'label'].values
x_train = x_train/255
x_test = test/255
x_train = x_train.reshape(-1,28,28,1)

In [ ]:
import matplotlib.pyplot as plt
# PREVIEW IMAGES
plt.figure(figsize=(15,4.5))
for i in range(30):  
    plt.subplot(3, 10, i+1)
    plt.imshow(x_train[i].reshape((28,28)),cmap=plt.cm.binary)
    plt.axis('off')
plt.subplots_adjust(wspace=-0.1, hspace=-0.1)
plt.show()

In [ ]:
x_train = x_train.reshape((-1,1,28,28))
x_test = x_test.values.reshape(-1,1,28,28)
x_train = np.stack((x_train,)*3, axis=1)
x_train = np.squeeze(x_train, axis=2)
x_train.shape


In [ ]:
x_test = np.stack((x_test,)*3, axis=1)
x_test = np.squeeze(x_test, axis=2)

In [ ]:
y_train = torch.from_numpy(y_train).type(torch.LongTensor)

x_train, x_test and y_train are made as tensors

In [ ]:
y_train

In [ ]:
Transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
BATCH = 32
epochs = 23
LR = 1e-3

Below code defines a custom PyTorch dataset class named GetData, which inherits from the Dataset class. This custom dataset class is designed to load data from a dataset for use with PyTorch models.

The __init__ method of the GetData class takes three arguments: x_train, y_train, and Transform. x_train is a tensor containing the input features for the dataset, y_train is a tensor containing the corresponding target labels for the dataset, and Transform is a PyTorch transform object that is applied to the input features during loading.

In the __init__ method, the input features and target labels are stored as instance variables self.X and self.Y, respectively. The transform object is also stored as an instance variable self.transform.

The __len__ method returns the number of samples in the dataset, which is the length of the input features tensor self.X.

The __getitem__ method is used to retrieve a single sample from the dataset given its index. In this method, the transform object is applied to the input features tensor self.X[index]. The resulting tensor is then permuted using the permute method to change its dimensions from (channels, height, width) to (height, width, channels). The contiguous method is called to ensure that the memory layout of the resulting tensor is contiguous. Finally, a tuple containing the transformed input features and the corresponding target label is returned.

In [ ]:
class GetData(Dataset):
    def __init__(self, x_train, y_train, Transform):
        self.X = x_train
        self.transform = Transform
        self.Y = y_train
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.transform(self.X[index]).permute((1, 2, 0)).contiguous(), self.Y[index]

In [ ]:
trainset = GetData(x_train,y_train,Transform)
trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True, num_workers=4)

In [ ]:
next(iter(trainloader))[0].shape

Below  code is creating nets number of convolutional neural network (CNN) models with the same architecture.

Each model is created using the nn.Sequential() module, which allows us to stack multiple layers together to form the model. The architecture of the model consists of several convolutional layers, activation functions (ReLU), batch normalization layers, dropout layers, and fully connected layers (linear).

The nn.Conv2d() module creates a 2-dimensional convolutional layer with a specified number of input channels (1), output channels (32, 64, or 128), kernel size (3 or 5), padding (1 or 2), and stride (2). The ReLU activation function is applied after each convolutional layer to introduce non-linearity, and the batch normalization layer is used to improve the convergence and generalization of the network. Dropout layers are added to prevent overfitting.

Finally, the model is optimized using the Adam optimizer and trained using the Cross-Entropy loss function. The optimizer is used to update the model's parameters during backpropagation, while the loss function calculates the difference between the predicted and actual class labels of the input data.






# Model of multiple CNNs

In [ ]:
model= nn.Sequential(
    nn.Conv2d(3,32,kernel_size =3, padding =1),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(32, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(32, 32, kernel_size=5, padding=2, stride=2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.4),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Conv2d(64, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Conv2d(64, 64, kernel_size=5, padding=2, stride=2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.4),
    nn.Conv2d(64, 128, kernel_size=4),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.Flatten(),
    nn.Dropout(0.4),
    nn.Linear(2048, 10)
    )
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

Below code represents a typical training loop in PyTorch for training a neural network model for a given number of epochs.

The outer loop iterates over the range of epochs.
tr_loss is the training loss that accumulates for each batch in the epoch.
correct and total keep track of the number of correctly predicted labels and the total number of labels in the epoch.
model.train() sets the model in training mode.
The inner loop iterates over the trainloader which loads the training data in batches. For each batch,
the inputs and labels are moved to the GPU device if available.
The forward pass is performed using the model on the input batch to obtain the outputs.
The loss is computed between the outputs and labels using a criterion such as cross-entropy.
The optimizer is zeroed out.
The backward pass is performed to compute the gradients and the optimizer is updated using these gradients to adjust the weights of the model.
The batch loss is accumulated to the epoch loss.
The predicted labels are computed by taking the argmax of the output probabilities.
correct and total are updated based on the number of correct predictions in the batch.
train_acc is computed by dividing the number of correctly predicted labels by the total number of labels and multiplying by 100 to obtain the percentage accuracy.
After the training epoch is completed, the model is set to evaluation mode using model.eval().
The validation loss is computed in a similar way to the training loss but using the validation dataset.
val_acc is computed in a similar way to train_acc.
Finally, the epoch number, epoch loss, train accuracy and validation accuracy are printed for the epoch.

# Training the model

In [ ]:
for epoch in range(epochs):
    tr_loss = 0.0
    correct = 0
    total = 0
    model = model.train()

    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(images)
        loss = criterion(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tr_loss += loss.detach().item()
        _,predicted = torch.max(outputs.data,1)
        total+=labels.size(0)
        correct += (predicted ==labels).sum().item()
        
    train_acc = 100 * correct/total

        
    val_loss = 0.0
    correct = 0
    total = 0
    model = model.eval()
    _, predicted = torch.max(outputs.data, 1)
    total+=labels.size(0)
    correct += (predicted ==labels).sum().item()
    val_acc = 100 * correct / total
    print('Epoch: %d | Loss: %.4f |Train accuracy=:%.5f, Validation accuracy=:%.5f' 
          %(epoch, tr_loss / i ,train_acc,val_acc))

In [ ]:
sample = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [ ]:
with torch.no_grad():
    model.eval()
    sample['Label'] = model(torch.from_numpy(x_test).to(DEVICE)).cpu().argmax(dim=1)

In [ ]:
sample.to_csv("submission.csv", index=False)